请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# **【2022 CCF BDCI 基于文心CV大模型的智慧城市视觉多任务识别】第3名方案**


依序运行全部代码块，即可完成训练、测试以及输出结果文件的过程。

模型对应的checkpoint文件已保存在目录下；其中，模型5基于GPNAS构造，训练较快且具有一致性，由于无一键导出参数功能，故此处未保存，可自行训练打印参数；其余事项请见下方特别注意！！！



**模型构建及调优**

1.模型总体介绍：

        解题思路：
    
            通过阅读相关文献定位到本任务是一个NAS领域的神经网络模型性能预测问题，但实际上我们并没有获得每个子网络的具体结构，只知道其所在的CV语境下的任务；
    
            此外，我们仅有500条训练数据，也就是说这还是一个小样本学习问题，所以我们考虑使用一些机器学习的手段辅以数据增强方法、以及CV语境先验知识来解决此任务。
    
        算法构建：

            首先，我们通过训练数据重采样，以及根据网络在CV任务中的语义信息，即人脸、人体、车辆和商品识别任务的相关性对各任务单独预测结果进行加权组合生成最终结果来提升模型性能以及泛化性，A榜分数提升至 0.78770625154；

            其次，学习CVPR2022 NAS Track2第一名的解决方案引入Stack Gradient Boosting方法与原模型进行集成综合来提升模型性能，A榜分数提升至 0.79883043865；
    
            再次，根据平台测评结果，从防止模型训练调参过拟合、提升模型泛化性能的角度出发，学习CVPR2022 NAS Track2第四名的解决方案，在部分任务上单独使用CatBoostRanker模型，并调整模型结构编码方式为demo所给出的编码方式以最终提升模型性能，A榜分数提升至 0.79973760966；
            
        代码组织结构：
            
            除GPNAS算法改进源码在另一文件 GPNAS.py 中外，其余代码均在文件 main.ipynb 中。
            
   算法结构框图：
        
   ![](https://ai-studio-static-online.cdn.bcebos.com/d4bab46696324d63bc932bed826634b296b4abf829b54ed7bfc488dc53a0cd7e)

2.数据增强策略：
     
         我们尝试过的数据增强策略有：数据融合、数据重采样、预测数据回填训练。
         
             （1）对于数据融合，我们曾尝试将原训练数据按任务相关性进行加权组合后的结果作为GPNAS模型训练数据，本地效果较差；此外，尝试将GPNAS在各任务单独预测模型中将同类型任务预测结果进行加权组合生成新的结果，测试效果较好，在最终模型5中采用将 veri、vehicleid、veriwild 三个任务单独训练并预测的结果按 0.04、0.92、0.04 的权重组合作为任务 vehicleid 的预测结果；
             
             （2）对于数据重采样，我们认为GPNAS模型在部分任务上初始化较差或欠拟合，故对训练数据进行重采样训练，部分参数下测试效果较好，在最终模型五中 vehicleid 任务的GPNAS模型采取重采样3次训练的策略；
             
             （3）对于预测数据回填训练，我们曾将全部八个任务的模型对于测试集的训练结果进行随机等距采样100条，作为模型的训练数据回填训练，而后再对测试集进行预测，本地尝试效果较好，A榜线上测试效果较差，且随间距选择不同精度波动较大，故暂时搁置，等待日后改进。     

3.调参优化策略：
    
         本地网格搜索交叉验证调参，选取本地测试精度较高若干组参数进行A榜线上测试其泛化能力，选取各任务精度较高模型、参数等进行组合，得到最终模型。
 
4.模型训练以及测试：
           
         详见相关代码块.

5.其它需要说明的内容：
        
         本最终模型由 Sub Model 1、Sub Model 2、Sub Model 3 组合而成；
             
         其中 Sub Model 1 用于任务 0、1、3、7，此模型源自 CVPR 2022 NAS Track2 Rank 1 解决方案，我们在其基础上进行了一些改进，原项目链接如下 https://aistudio.baidu.com/aistudio/projectdetail/3751972?channel=0&channelType=0&sUid=2709743&shared=1&ts=1668438343541
             
         其中 Sub Model 2 用于任务 2、4、6，此模型源自 CVPR 2022 NAS Track2 Rank 4 解决方案，我们在其基础上进行了一些改进，项目链接如下 https://aistudio.baidu.com/aistudio/clusterprojectdetail/4051842
             
         其中 Sub Model 3 用于任务 5， 此模型为对官方基线demo以及GPNAS方法的改进。
 
 6.特别注意：
         
         本团队所提交的最高得分结果中，产生任务 0、1、3、7 预测结果的 Sub Model 1 运行环境为 PaddlePaddle 2.2.2，本项目基于 PaddlePaddle 2.3.2，在此环境下模型经训练并测试所产生的结果略有不用，若想完全复现本团队最高得分结果，请修改项目环境为 PaddlePaddle 2.2.2，以获取对应任务预测结果并与其他任务预测结果进行组合；
         
         此外，其余产生任务 2、4、5、6 预测结果的模型运行环境均为 PaddlePaddle 2.3.2，经本团队测试，其在 PaddlePaddle 2.2.2 环境下运行结果亦会产生差别，请特别注意；
         
         总结而言，若想完全复现本团队最高分结果，请在 PaddlePaddle 2.2.2 对任务 0、1、3、7 进行预测，在 PaddlePaddle 2.3.2 对任务 2、4、5、6 进行预测，而后将结果进行组合，本项目基于 PaddlePadle 2.3.2，若预测结果经B榜测试分数有差异且会造成名次影响，请按上述流程进行重新获取结果，敬请谅解；
         
         本项目只保存了 PaddlePaddle 2.3.2 环境下模型 Checkpoint 文件。

# **导入必要的包**

In [ ]:
import catboost
import lightgbm
import xgboost
import sklearn
from sklearn import ensemble
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor, StackingRegressor
import numpy as np
import scipy
import scipy.stats
from scipy.stats import rankdata
import json
from catboost import CatBoostRanker
from xgboost import XGBRegressor
from sklearn.base import RegressorMixin, BaseEstimator
from GPNAS import GPNAS

# **数据导入及处理**

In [ ]:
# 编码方式1
def convert_X_v1(arch_str):
    temp_arch = []
    for i,elm in enumerate(arch_str):
        if i in [3,6,9,12,15,18,21,24,27,30,33,36]: pass #Get rid of non-info columns,all is 768
        elif elm == 'j': temp_arch.append(1-2) #Transform it to number then central data, normalize Data
        elif elm == 'k': temp_arch.append(2-2) #Transform it to number then central data, normalize Data
        elif elm == 'l': temp_arch.append(3-2)  #Transform it to number then central data, normalize Data
        elif int(elm) == 0: temp_arch.append(2-2)  #Make 0 as 2 as it should contain neutral information（reduce correlation), then central data, normalize Data
        else: temp_arch.append(int(elm)-2)  #central data,normalize Data
    return(temp_arch)

# 编码方式2
def convert_X_v2(arch_str):
    temp_arch = []
    total_1 = 0
    total_2 = 0
    ts = ''
    for i in range(len(arch_str)):
        if i % 3 != 0 and i != 0 and i <= 30:
            elm = arch_str[i]
            ts = ts + elm
            if elm == 'l' or elm == '1':
                temp_arch = temp_arch + [1, 1, 0, 0]
            elif elm == 'j' or elm == '2':
                temp_arch = temp_arch + [0, 1, 1, 0]
            elif elm == 'k' or elm == '3':
                temp_arch = temp_arch + [0, 0, 1, 1]
            else:
                temp_arch = temp_arch + [0, 0, 0, 0]

        elif i % 3 != 0 and i != 0 and i > 30:
            elm = arch_str[i]
            if elm == 'l' or elm == '1':
                temp_arch = temp_arch + [1, 1, 0, 0, 0]
            elif elm == 'j' or elm == '2':
                temp_arch = temp_arch + [0, 1, 1, 0, 0]
            elif elm == 'k' or elm == '3':
                temp_arch = temp_arch + [0, 0, 1, 1, 0]
            else:
                temp_arch = temp_arch + [0, 0, 0, 0, 1]

    return temp_arch


with open('./CCF_UFO_train.json', 'r') as f:
    train_data = json.load(f)

with open('./CCF_UFO_test.json', 'r') as f:
    test_data = json.load(f)

test_arch_list_v1 = []
test_arch_list_v2 = []
for key in test_data.keys():
    test_arch = convert_X_v1(test_data[key]['arch'])
    test_arch_list_v1.append(test_arch)
    test_arch = convert_X_v2(test_data[key]['arch'])
    test_arch_list_v2.append(test_arch)

train_list = [[], [], [], [], [], [], [], []]
arch_list_train_v1 = []
arch_list_train_v2 = []
name_list = ['cplfw_rank', 'market1501_rank', 'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank', 'veriwild_rank', 'sop_rank']

for key in train_data.keys():
    for idx, name in enumerate(name_list):
        train_list[idx].append(train_data[key][name])
    arch_list_train_v1.append(convert_X_v1(train_data[key]['arch']))
    arch_list_train_v2.append(convert_X_v2(train_data[key]['arch']))

Y_all_int = np.array(train_list)
Y_all_log = np.log((Y_all_int + 1)/(500 - Y_all_int))  # Transfer rank data by Sigmoid function

# **Sub Model 1**

In [ ]:
max_iter = [10000,10000,10000,10000,10000,10000,10000,10000]
learning_rate = [0.005,0.038,0.035,0.03,0.025,0.01,0.03,0.01]
max_depth = [1,3,2,2,2,3,1,3]  # depth for GBRT(huber), CATGB(MSE), GBRT2(MSE), CATGB2(huber)
max_depth2 = [1,1,1,1,1,1,1,1]  # depth for HISTGB, LIGHTGB, XGB

model_GBRT = []
model_HISTGB = []
model_CATGB = []
model_LIGHTGB = []
model_XGB = []
model_GBRT2 = []
model_CATGB2= []

for i in range(8):

    para_GBRT = {
        "n_estimators": max_iter[i],
        "max_depth": max_depth[i],
        "subsample": .8,
        "learning_rate": learning_rate[i],
        "loss": 'huber',
        "max_features": 'sqrt',
        "random_state": 1
        }
    model_GBRT.append(ensemble.GradientBoostingRegressor(**para_GBRT)) 

    para_HISTGB = {
        "max_depth": max_depth2[i],
        "max_iter": max_iter[i],
        "learning_rate": learning_rate[i],
        "loss": 'least_squares',
        "max_leaf_nodes": 31,
        "min_samples_leaf": 5,
        "l2_regularization": 5,
        "random_state": 1
        }
    model_HISTGB.append(HistGradientBoostingRegressor(**para_HISTGB))

    model_CATGB.append(catboost.CatBoostRegressor(
                            iterations = max_iter[i],
                            learning_rate = learning_rate[i],
                            depth = max_depth[i],
                            silent = True,
                            task_type = "CPU",
                            loss_function = 'RMSE',
                            eval_metric = 'RMSE',
                            random_seed = 1,
                            od_type ='Iter',
                            metric_period = 75,
                            od_wait = 100
                            ))

    model_LIGHTGB.append(lightgbm.LGBMRegressor(
                            boosting_type = 'gbdt',
                            learning_rate = learning_rate[i],
                            num_leaves = 31,
                            max_depth = max_depth2[i],
                            alpha = 0.1,
                            n_estimators = max_iter[i],
                            random_state = 1
                            ))

    model_XGB.append(xgboost.XGBRegressor(
                            learning_rate = learning_rate[i],
                            tree_method = 'auto',
                            max_depth = max_depth2[i],
                            alpha = 0.8,
                            n_estimators = max_iter[i],
                            random_state = 1
                            ))

    para_GBRT2 = {
        "n_estimators": max_iter[i],
        "max_depth": max_depth[i],
        "subsample": .8,
        "learning_rate": learning_rate[i],
        "loss": 'ls',
        "max_features": 'log2',
        "random_state": 1
        } 
    model_GBRT2.append(ensemble.GradientBoostingRegressor(**para_GBRT2))

    model_CATGB2.append(catboost.CatBoostRegressor(
                            iterations = max_iter[i],
                            learning_rate = learning_rate[i],
                            depth = max_depth[i],
                            silent = True,
                            task_type = "CPU",
                            loss_function = 'Huber:delta=2',
                            eval_metric = 'Huber:delta=2',
                            random_seed = 1,
                            od_type = 'Iter',
                            metric_period = 75,
                            od_wait = 100,
                            l2_leaf_reg = 1,
                            subsample = 0.8
                            ))

list_est = []

for i in range(8): 
    list_est.append([
    ('GBRT', model_GBRT[i]),
    ('HISTGB', model_HISTGB[i]),
    ('CATGB', model_CATGB[i]),
    ('LIGHTGB', model_LIGHTGB[i]),
    ('XGB', model_XGB[i]),
    ('GBRT2', model_GBRT2[i]),
    ('CATGB2', model_CATGB2[i])
    ])

# **Sub Model 2**

In [ ]:
para_catboost = [
    {'depth': 4, 'iterations': 531, 'learning_rate': 0.023319003825470877, 'l2_leaf_reg': 0.47729814558364875,
        'loss_function': 'PairLogit', 'thread_count': 1, 'verbose': False},
    {'depth': 2, 'iterations': 880, 'learning_rate': 0.22190109268672867, 'l2_leaf_reg': 1.3682444269346437,
        'bagging_temperature': 0.5837541985172918, 'loss_function': 'PairLogit', 'boosting_type': 'Plain',
        'bootstrap_type': 'MVS', 'thread_count': 1, 'verbose': False},
    {'depth': 2, 'iterations': 948, 'learning_rate': 0.3502492588081655, 'l2_leaf_reg': 3.14720732769261e-06,
        'bagging_temperature': 0.6942886010018183, 'loss_function': 'PairLogit', 'boosting_type': 'Plain',
        'bootstrap_type': 'Bayesian', 'thread_count': 1, 'verbose': False},
    {'depth': 1, 'iterations': 928, 'learning_rate': 0.8832026427379356, 'l2_leaf_reg': 6.9569849693551395,
        'bagging_temperature': 2.342749037758048, 'random_strength': 0.04228744015207187, 'loss_function': 'PairLogit',
        'thread_count': 1, 'verbose': False},
    {'depth': 2, 'iterations': 968, 'learning_rate': 0.6088070150760077, 'l2_leaf_reg': 0.03446304987797686,
        'loss_function': 'PairLogit', 'thread_count': 1, 'verbose': False},
    {'depth': 1, 'iterations': 983, 'learning_rate': 0.3685720989187422, 'l2_leaf_reg': 0.013573391424731586,
        'loss_function': 'PairLogit', 'thread_count': 1, 'verbose': False},
    {'depth': 1, 'iterations': 788, 'learning_rate': 0.6771701021511635, 'l2_leaf_reg': 40.414268221784425,
        'bagging_temperature': 0.8256212472915649, 'loss_function': 'PairLogit', 'boosting_type': 'Plain',
        'bootstrap_type': 'MVS', 'thread_count': 1, 'verbose': False},
    {'depth': 1, 'iterations': 993, 'learning_rate': 0.13545368910458658, 'l2_leaf_reg': 1.080898618076245,
        'loss_function': 'PairLogit', 'thread_count': 1, 'verbose': False},
    ]


class CatBoostPairwiseRanker(RegressorMixin, CatBoostRanker):

    def fit(self, X, y=None, group_id=None, **kwargs):
        group_id = np.ones_like(y).astype(int)
        self._loss_value_change = np.ones(X.shape[1])
        
        return super().fit(X, y, group_id, **kwargs)

    def predict(self, X, **kwargs):
        return super().predict(X, **kwargs)


# **Sub Model 3**

In [ ]:
class GPNAS_MIX(object):

    def __init__(self, cov_w=None, w=None, c_flag=2, m_flag=2, hp_mat=0.0000001, hp_cov=0.01, icov=0):
        self.sublist = []
        for i in range(3):
            self.sublist.append(GPNAS(cov_w=cov_w, w=w, c_flag=c_flag, m_flag=m_flag, hp_mat=hp_mat, hp_cov=hp_cov, icov=icov))
        
    def fit(self, X_train, Y_train_all):
        train_num = [400, 500, 500]
        train_times = [1, 3, 1]

        for i in range(3):
            X_train_k, Y_train_k = X_train[0:train_num[i]:1], Y_train_all[i + 4][0:train_num[i]:1]
            
            self.sublist[i].get_initial_mean(X_train_k[0::1], Y_train_k[0::1])
            init_cov = self.sublist[i].get_initial_cov(X_train_k)

            for j in range(train_times[i]):
                self.sublist[i].get_posterior_mean(X_train_k[1::1], Y_train_k[1::1])

    def predict(self, X_test, X_train, Y_train_all):
        rank_t = []
        for i in range(3):
            rank_t.append(self.sublist[i].get_predict_jiont(X_test, X_train, Y_train_all[i + 4]))

        return rank_t[0]*0.04 + rank_t[1]*0.92 + rank_t[2]*0.04

# **各任务 模型及数据编码 集成**

In [ ]:
Model_list = []
Model_list.append(StackingRegressor(estimators=list_est[0], final_estimator=GPNAS(c_flag=2, m_flag=2, hp_mat=0.5, hp_cov=3, icov=1), passthrough=False, n_jobs=-1))
Model_list.append(StackingRegressor(estimators=list_est[1], final_estimator=GPNAS(c_flag=2, m_flag=2, hp_mat=0.5, hp_cov=3, icov=1), passthrough=False, n_jobs=-1))
Model_list.append(CatBoostPairwiseRanker(**para_catboost[2]))
Model_list.append(StackingRegressor(estimators=list_est[3], final_estimator=GPNAS(c_flag=2, m_flag=2, hp_mat=0.5, hp_cov=3, icov=1), passthrough=False, n_jobs=-1))
Model_list.append(CatBoostPairwiseRanker(**para_catboost[4]))
Model_list.append(GPNAS_MIX(cov_w=None, w=None, c_flag=2, m_flag=2, hp_mat=0.5, hp_cov=0.01, icov=0))
Model_list.append(CatBoostPairwiseRanker(**para_catboost[6]))
Model_list.append(StackingRegressor(estimators=list_est[7], final_estimator=GPNAS(c_flag=2, m_flag=2, hp_mat=0.5, hp_cov=3, icov=1), passthrough=False, n_jobs=-1))

X_test, X_train, Y_train = [], [], []
for i in range(8):
    if i in [0, 1, 3, 7]:
        X_test.append(np.array(test_arch_list_v1))
        X_train.append(np.array(arch_list_train_v1))
        Y_train.append(Y_all_log[i])
    else: # 2 4 5 6
        X_test.append(np.array(test_arch_list_v2))
        X_train.append(np.array(arch_list_train_v2))
        Y_train.append(Y_all_int[i])

# **模型训练以及预测**

In [ ]:
rank_all= []

for i in range(8):
    print('The task:', name_list[i])
    
    #此处可将checkpoint文件模型参数导入，免去训练时间
    print("~~fitting")
    if i == 5:
        Model_list[i].fit(X_train[i], Y_train)
    else:
        Model_list[i].fit(X_train[i], Y_train[i])

    print("~~predicting")
    if i == 5:
        rank_all.append(Model_list[i].predict(X_test[i], X_train[i], Y_train))
    elif i in [0, 1, 3, 7]:
        rank_all.append(np.round((X_test[i].shape[0] - 1)/(1 + np.exp(-1 * Model_list[i].predict(X_test[i])))))
    else: # 2 4 6
        rank_all.append(rankdata(Model_list[i].predict(X_test[i])) - 1)

    print('\n')

# **模型checkpoint保存**

In [ ]:
#若想自行生成并保存checkpoint文件可将下方代码解除注释，正常预测结果的生成不需要此步骤

# import joblib

# joblib.dump(Model_list[0], "./Model_list_0.m")
# joblib.dump(Model_list[1], "./Model_list_1.m")
# joblib.dump(Model_list[2], "./Model_list_2.m")
# joblib.dump(Model_list[3], "./Model_list_3.m")
# joblib.dump(Model_list[4], "./Model_list_4.m")
# joblib.dump(Model_list[6], "./Model_list_6.m")
# joblib.dump(Model_list[7], "./Model_list_7.m")

# 模型5基于GPNAS构造，训练较快且具有一致性，由于无一键导出参数功能，故此处未保存，可自行训练打印参数。

# **预测结果生成及保存**

In [ ]:
for idx,key in enumerate(test_data.keys()):
    test_data[key]['cplfw_rank'] = int(rank_all[0][idx])
    test_data[key]['market1501_rank'] = int(rank_all[1][idx])
    test_data[key]['dukemtmc_rank'] = int(rank_all[2][idx])
    test_data[key]['msmt17_rank'] = int(rank_all[3][idx])
    test_data[key]['veri_rank'] = int(rank_all[4][idx])
    test_data[key]['vehicleid_rank'] = int(rank_all[5][idx])
    test_data[key]['veriwild_rank'] = int(rank_all[6][idx])
    test_data[key]['sop_rank'] = int(rank_all[7][idx])
print('Ready to save results!')

with open('./CCF_UFO_submit_B_Rank3.json', 'w') as f:
    json.dump(test_data, f)